# LightFM model for influence marketing

Description

### Model theoretical explanation
This model is based in ...

### 1. Import Libraries

In [19]:
# Install all the libraries in requirements.txt
import sys
import os

import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scrapbook as sb

import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation

# Import LightFM's evaluation metrics
from lightfm.evaluation import precision_at_k as lightfm_prec_at_k
from lightfm.evaluation import recall_at_k as lightfm_recall_at_k

# Import repo's evaluation metrics
from recommenders.evaluation.python_evaluation import precision_at_k, recall_at_k

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.models.lightfm.lightfm_utils import (
    track_model_metrics, prepare_test_df, prepare_all_predictions,
    compare_metric, similar_users, similar_items)

print("System version: {}".format(sys.version))
print("LightFM version: {}".format(lightfm.__version__))

System version: 3.8.5 (default, Sep  4 2020, 02:22:02) 
[Clang 10.0.0 ]
LightFM version: 1.16


### 2. Defining Variables

In [22]:
# Select MovieLens data size
MOVIELENS_DATA_SIZE = '100k'

# default number of recommendations
K = 10
# percentage of data used for testing
TEST_PERCENTAGE = 0.25
# model learning rate
LEARNING_RATE = 0.25
# no of latent factors
NO_COMPONENTS = 20
# no of epochs to fit model
NO_EPOCHS = 20
# no of threads to fit model
NO_THREADS = 32
# regularisation for both user and item features
ITEM_ALPHA = 1e-6
USER_ALPHA = 1e-6

# seed for pseudonumber generations
SEED = 42

In [28]:
df = pd.read_csv('/Users/joaquincasasus/Desktop/Influencer_marketing/influence_marketing/influencers_df.csv')


### 3. Retrive Data

In [23]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    genres_col='genre',
    header=["userID", "itemID", "rating"]
)
# quick look at the data
data.sample(5, random_state=SEED)

100%|██████████| 4.81k/4.81k [00:54<00:00, 88.9KB/s]


,userID,itemID,rating,genre
75721,498,693,3.0,Drama
80184,642,542,5.0,Animation|Children's|Musical|Romance
19864,58,135,4.0,Drama|Mystery|Sci-Fi|Thriller
76699,495,674,3.0,Horror
92991,618,735,3.0,Drama


In [30]:
dataset = Dataset()

In [31]:
df

,Account,Link,Followers,Audience Country,Authentic engagement,Engagement avg,Category1,Hashtags,Cost Story,Cost Post
0,cristiano,https://www.instagram.com/cristiano/,400100000.0,India,7800000.0,9500000.0,Sports with a ball,"['#fashionblogger', '#baby', '#photographer', ...",468000.0,1092000.0
1,kyliejenner,https://www.instagram.com/kyliejenner/,308800000.0,United States,6200000.0,10100000.0,Fashion,"['#music', '#instagood', '#soccer', '#dog']",372000.0,868000.0
2,leomessi,https://www.instagram.com/leomessi/,306300000.0,Argentina,4800000.0,6500000.0,Sports with a ball,['#flex'],288000.0,672000.0
3,kendalljenner,https://www.instagram.com/kendalljenner/,217800000.0,United States,3400000.0,5400000.0,Modeling,"['#blessed', '#beautyful', '#gains', '#funny',...",204000.0,476000.0
4,selenagomez,https://www.instagram.com/selenagomez/,295800000.0,United States,2700000.0,3600000.0,Music,['#travel'],162000.0,378000.0
...,...,...,...,...,...,...,...,...,...,...
995,senoritasaeva,https://www.instagram.com/senoritasaeva/,7700000.0,Russia,246600.0,318200.0,Lifestyle,"['#foodislifee', '#gym', '#life']",14796.0,34524.0
996,manuelneuer,https://www.instagram.com/manuelneuer/,11500000.0,Germany,146500.0,210200.0,Sports with a ball,"['#surfing', '#healthy', '#smilemore', '#boxin...",8790.0,20510.0
997,sahilkhan,https://www.instagram.com/sahilkhan/,10100000.0,India,176500.0,239800.0,Fitness,"['#foodlife', '#like4like', '#beautyblogger', ...",10590.0,24710.0
998,mohanshakti,https://www.instagram.com/mohanshakti/,13700000.0,India,146400.0,175500.0,Art,"['#like4like', '#sun', '#followback', '#picoft...",8784.0,20496.0
